In [1]:
import requests
import pandas as pd

Read in BM unit data...

In [2]:
web = 'https://data.elexon.co.uk/bmrs/api/v1/reference/bmunits/all'
r = requests.get(web)
df = pd.DataFrame(r.json())
df.set_index('nationalGridBmUnit', inplace=True)
# last updated Jan 2023
df_ids = pd.read_csv('../data/ids.csv', index_col=0)
# last updated Jan 2023
df_loc = pd.read_csv('../data/plant-locations.csv', index_col=0)
lon = []
lat = []
for BMU in df.index:
    try:
        # creating and passing series to new column
        new = df_ids.loc[df_ids['ngc_bmu_id'].fillna('').str.contains(BMU)]
        lon.append(df_loc.loc[new.index.values[0]]['longitude'])
        lat.append(df_loc.loc[new.index.values[0]]['latitude'])
    except:
        lon.append('')
        lat.append('')
df['longitude'] = lon
df['latitude'] = lat

df

,elexonBmUnit,fuelType,leadPartyName,bmUnitType,fpnFlag,longitude,latitude
nationalGridBmUnit,,,,,,,
ABERU-1,E_ABERDARE,None,UK Power Reserve Limited,E,True,-3.44926,51.711177
ABRBO-1,T_ABRBO-1,WIND,Aberdeen Offshore Wind Farm,T,True,-1.9728,57.2226
ABRTW-1,E_ABRTW-1,WIND,Npower Commercial Gas Limited,E,True,-3.988675,55.621764
ACHLW-1,C__PSMAR001,WIND,Smartestenergy Limited,S,True,-3.4565,58.4479
ACHRW-1,T_ACHRW-1,WIND,Mobius Renewables Gen. (GB2,T,True,-5.393722,56.34286
...,...,...,...,...,...,...,...
YEP-SWT01,2__MSWEB000,None,EDF Energy Customers Ltd,G,True,,
YEP-SYM01,2__MSYMB000,None,E.ON Next Energy Ltd,G,True,,
YEP-TFE01,2__MTGPL000,None,TotalEnergies Gas & Power Ltd,G,True,,


Read in FPN data...

In [3]:
# can get data from most recent settlement period, but limited by demand +1H after
end_date = pd.Timestamp.now()
settlementDate = (end_date - pd.Timedelta(hours=0.5)).date()
settlementPeriod = (end_date - pd.Timedelta(hours=0.5)).hour * 2

# options - 'PN', 'QPN', 'MILS', 'MELS
dataset = 'PN'

web = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset=' + dataset + '&settlementDate=' + str(settlementDate) + '&settlementPeriod=' + str(settlementPeriod) + '&format=json'

r = requests.get(web)
df_FPN = pd.DataFrame(r.json()['data'])
# df.index = df['nationalGridBmUnit']
df_FPN.set_index('nationalGridBmUnit', inplace=True)
df_FPN.rename(columns={'levelTo':'FPN'}, inplace=True)
df_FPN

,dataset,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,FPN,bmUnit
nationalGridBmUnit,,,,,,,,
ARNKB-1,PN,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,-12,0,E_ARNKB-1
CRSSB-1,PN,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,-1,0,E_CRSSB-1
BHLAW-1,PN,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,97,98,T_BHLAW-1
CLDNW-1,PN,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,169,168,T_CLDNW-1
GRGBW-1,PN,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,111,105,T_GRGBW-1
...,...,...,...,...,...,...,...,...
AG-FLX02N,PN,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NFLEX001
AG-FLX07N,PN,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NFLEX002
AG-HEL0DN,PN,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NHABI005


Read in maximum export level data...

In [4]:
# options - 'PN', 'QPN', 'MILS', 'MELS'
dataset = 'MELS'

web = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset=' + dataset + '&settlementDate=' + str(settlementDate) + '&settlementPeriod=' + str(settlementPeriod) + '&format=json'

r = requests.get(web)
df_mel = pd.DataFrame(r.json()['data'])
# df.index = df['nationalGridBmUnit']
df_mel.set_index('nationalGridBmUnit', inplace=True)
df_mel.rename(columns={'levelTo':'MEL'}, inplace=True)
df_mel

,dataset,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,MEL,bmUnit
nationalGridBmUnit,,,,,,,,
DAMC-1,MELS,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,763,762,T_DAMC-1
DIDCB6,MELS,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,333,332,T_DIDCB6
LAGA-1,MELS,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,392,391,T_LAGA-1
PEMB-21,MELS,2024-02-03,28,2024-02-03T13:59:00Z,2024-02-03T14:00:00Z,438,437,T_PEMB-21
AG-ASTK05,MELS,2024-02-03,28,2024-02-03T13:58:00Z,2024-02-03T14:00:00Z,50,50,2__ASTAT005
...,...,...,...,...,...,...,...,...
AG-FLX02N,MELS,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NFLEX001
AG-FLX07N,MELS,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NFLEX002
AG-HEL0DN,MELS,2024-02-03,28,2024-02-03T13:30:00Z,2024-02-03T14:00:00Z,0,0,V__NHABI005


Combine dataframes...

In [6]:
df = df[~df.index.duplicated(keep='first')]
df_FPN = df_FPN[~df_FPN.index.duplicated(keep='first')]
df_mel = df_mel[~df_mel.index.duplicated(keep='first')]
result = pd.concat([df, df_FPN, df_mel], axis=1, join='inner')
result = result[['settlementDate', 'settlementPeriod', 'fuelType', 'longitude', 'latitude', 'FPN', 'MEL']]

df_gen = result.loc[result['FPN'] > 0]
df_gen

,settlementDate,settlementDate,settlementPeriod,settlementPeriod,fuelType,longitude,latitude,FPN,MEL
nationalGridBmUnit,,,,,,,,,
ABRBO-1,2024-02-03,2024-02-03,28,28,WIND,-1.9728,57.2226,53,97
ABRTW-1,2024-02-03,2024-02-03,28,28,WIND,-3.988675,55.621764,36,36
ACHRW-1,2024-02-03,2024-02-03,28,28,WIND,-5.393722,56.34286,43,42
AFTOW-1,2024-02-03,2024-02-03,28,28,WIND,-4.1691,55.3128,42,50
AG-FLX01P,2024-02-03,2024-02-03,28,28,None,,,2,2
...,...,...,...,...,...,...,...,...,...
WLNYO-4,2024-02-03,2024-02-03,28,28,WIND,-3.522,54.044,257,322
WLNYW-1,2024-02-03,2024-02-03,28,28,WIND,-3.522,54.044,89,159
WTMSO-1,2024-02-03,2024-02-03,28,28,WIND,0.681152,53.943155,191,204


In [8]:
df_gen['FPN'].sum()

30581

Read in the demand...

https://bmrs.elexon.co.uk/total-load-actual-and-day-ahead - includes transmission and distributed network connected load and is actual. (code for this below)

https://bmrs.elexon.co.uk/rolling-system-demand - elexon calls this the rolling system demand, but it is just the sum of transmission connected generation.

In [10]:
import requests
import pandas as pd

# Actual total load per bidding zone per Settlement Period, the total load being defined as equal
# to the sum of power generated by plants on both Transmission System Operators/Distribution System
# Operators networks, from which is deduced:
# - The balance (export-import) of exchanges on interconnections between neighbouring bidding zones.
# - The power absorbed by energy storage resources.

# demand data is published +1H after
from_ = (end_date).date()
to_ = from_

web = 'https://data.elexon.co.uk/bmrs/api/v1/demand/actual/total?from=' + str(from_) + '&to=' + str(to_) + '&settlementPeriodFrom=' + str(settlementPeriod) + '&settlementPeriodTo=' + str(settlementPeriod) + '&format=json'
r = requests.get(web)
df = pd.DataFrame(r.json()['data'])
df

,publishTime,startTime,settlementDate,settlementPeriod,quantity
0,2024-02-03T14:25:10Z,2024-02-03T13:30:00Z,2024-02-03,28,39599.0
